In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from sklearn.model_selection import train_test_split

print("PyTorch Version:", torch.__version__)
print("Torchvision Version: at least 0.4.0")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the path to the dataset directory
data_directory = "/kaggle/input/stanford-dogs-dataset/images/Images"

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

images_list = []
labels_list = []
label_dict = {}

for idx, breed in enumerate(os.listdir(data_directory)[:50]): 
    breed_directory = os.path.join(data_directory, breed)
    label_dict[breed] = idx
    for image_name in os.listdir(breed_directory):
        image_path = os.path.join(breed_directory, image_name)
        image = Image.open(image_path).convert('RGB')
        image = data_transform(image)
        images_list.append(image)
        labels_list.append(label_dict[breed])

images_tensor = torch.stack(images_list)
labels_tensor = torch.tensor(labels_list)

X_train, X_test, y_train, y_test = train_test_split(images_tensor, labels_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, len(set(labels_list)))
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=3)

train_losses = []
train_accuracies = []
for epoch in range(30):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)
    scheduler.step(train_losses[-1]) 
    print(f'Training - Epoch {epoch + 1}, Loss: {train_losses[-1]}, Accuracy: {train_accuracies[-1]} %')

plt.figure(figsize=(10, 5))
plt.plot(range(1, 31), train_accuracies, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training Accuracy vs Epochs')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range(1, 31), train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs Epochs')
plt.legend()
plt.show()

model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Testing - Loss: {test_loss / len(test_loader)}, Accuracy: {100 * correct / total} %')

def predict_image(image_tensor):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.unsqueeze(0).to(device)
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return predicted.item()

random_index = np.random.randint(len(X_test))
random_image = X_test[random_index]
random_label = y_test[random_index]

predicted_label = predict_image(random_image)

print(f"Correct Label: {list(label_dict.keys())[random_label]}")
print(f"Predicted Label: {list(label_dict.keys())[predicted_label]}")



In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pth')

In [ ]:
def visualize_image(image, correct_label, predicted_label):
    image = image.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)

    plt.imshow(image)
    plt.axis('off')

    plt.title(f"Correct Label: {list(label_dict.keys())[correct_label]}\nPredicted Label: {list(label_dict.keys())[predicted_label]}")

random_index = np.random.randint(len(X_test))
random_image = X_test[random_index]
random_label = y_test[random_index]

predicted_label = predict_image(random_image)

plt.figure(figsize=(5, 5))
visualize_image(random_image, random_label, predicted_label)
plt.show()
